In [3]:
ua.random

FakeUserAgentError: Error occurred during getting browser

In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import requests
import json
import os
from lxml import etree
import urllib
import time
from datetime import datetime
from selenium import webdriver
from random import randint
from fake_useragent import UserAgent
from tqdm import tqdm
import pickle

In [ ]:
category_url = 'https://themeforest.net/category'
ua = UserAgent()

now = datetime.now()

headers = {
    'User-Agent': 'Mozilla/5.0 (iPad; U; CPU OS 3_2_1 like Mac OS X; en-us) AppleWebKit/531.21.10 (KHTML, like Gecko) Mobile/7B405',
    'Content-Type': 'text/html'}

response_cat = requests.get(category_url, headers = headers, timeout = 1)

cookies = response_cat.cookies
soup = bs(response_cat.content, 'lxml')

## Create Main DF
theme_df = pd.DataFrame(columns = ['datetime', 'prod_name', 'sort_type', 'prod_rank', 'prod_url', 'prod_author', 
                                  'prod_cat', 'prod_price', 'prod_rating', 'prod_sales', 'prod_details', 
                                  'main_cat'])
    
## Retrieve categories from base url
first_categories = soup.find_all(attrs = {'id': 'first'})
links = first_categories[0].find_all('a')


## Append links to list
category_url_ls = []
category_name_ls = []
for link in links:
    if link.get('href').split('/')[-2] == 'category':
        category_url_ls.append(link.get('href'))
        category_name_ls.append(link.get('href').replace('/category/',''))



browser = webdriver.Chrome()

base_url = 'https://themeforest.net'

sort_dict = {'best_sellers': '&referrer=search&sort=sales&utf8=✓&view=list',
             'trending': '&referrer=search&sort=trending&utf8=✓&view=list',
             'highest_rating': '&referrer=search&sort=rating&utf8=✓&view=list',
             'low_to_high_price': '&referrer=search&sort=price-asc&utf8=✓&view=list',
             'high_to_low_price': '&referrer=search&sort=price-desc&utf8=✓&view=list'}

master_ls = []

## iterate through categories retrieved
for c, cat in enumerate(category_url_ls):
    
    cat_url = base_url + cat
    cat_name = category_name_ls[c]
    print('Starting iteration {} for category {} now!'.format(c + 1, cat_name))
    ## iterate through sort methods
    for sort_key, sort_url in sort_dict.items():
    
        ## iterate through pages
        for i in tqdm(range(1,61)):

            cat_page_url = cat_url + '?page={}'.format(i) + sort_url
            
            time.sleep(randint(1,3))
            
            browser.get(cat_page_url)
                        
            cat_response = browser.page_source
            
            cat_soup = bs(cat_response, 'lxml')

            ## retrieve headings
            prod_heading = cat_soup.find_all('h3', attrs = {'class': 'product-list__heading'})

            ## extract name, url, and rank from heading
            prod_names_ls = []
            prod_urls_ls = []
            prod_ranks_ls = []
            for prod in prod_heading:
                temp_a = prod.find('a')
                prod_url = 'https://themeforest.net' + temp_a.get('href')
                prod_rank = prod_url.split('rank=')[1]
                prod_name = (temp_a.getText())

                prod_names_ls.append(prod_name)
                prod_urls_ls.append(prod_url)
                prod_ranks_ls.append(prod_rank)

            ## retrieve category path
            prod_meta_cat = cat_soup.find_all('span', attrs = {'class': 'meta-categories -no-slash'})

            ## extract category path
            prod_category_ls = []
            for meta_cat in prod_meta_cat:
                temp_ls = []
                for b_tag in meta_cat.find_all('b'):
                    temp_ls.append(b_tag.getText())

                prod_category_ls.append(temp_ls)   

            ## retrieve product details
            prod_details = cat_soup.find_all(attrs = {'class': 'product-list__column-category'})

            ## retrieve product details
            prod_details_ls = []
            for prod_detail in prod_details:
                temp_details = []
                for br, br_tag in enumerate(prod_detail.find('p').getText().split('\n')):
                    if len(br_tag) > 10:
                        temp_details.append(br_tag.replace('    ', ''))
                prod_details_ls.append(temp_details)

            ## retrieve product authors
            prod_author = cat_soup.find_all('div', attrs = {'class': 'product-list__info-author'})

            ## extract authors
            prod_authors_ls = []
            for author in prod_author:
                prod_authors_ls.append(author.find('a').getText())

            ## retrieve product prices
            prod_price = cat_soup.find_all('div', attrs = {'class': 'product-list__column-price'})

            ## extract product prices
            prod_prices_ls = []
            for price in prod_price:
                prod_prices_ls.append(price.find('p').getText().replace('$',''))

            ## retrieve product ratings and sales
            prod_num_ratings = cat_soup.find_all(attrs = {'class': 'product-list__info-desktop'})

            ## extract ratings and sales
            prod_num_ratings_ls = []
            prod_num_sales_ls = []
            for num_rating in prod_num_ratings:
                temp_num_rating = ''
                temp_num_sales = ''
                for text in num_rating.getText().split('\n'):
                    text = text.replace(' ', '')
                    if len(text) > 0:
                        if 'ratings' in text:
                            temp_num_rating = text.split('r')[0]
                        if 'Sales' in text:
                            temp_num_sales = text.split('S')[0]
                prod_num_ratings_ls.append(temp_num_rating)
                prod_num_sales_ls.append(temp_num_sales)

            temp_df = pd.DataFrame({'datetime': now, 'prod_name': prod_names_ls, 'sort_type': sort_key, 'prod_rank': prod_ranks_ls, 
                 'prod_url': prod_urls_ls, 'prod_author': prod_authors_ls, 'prod_cat': prod_category_ls,
                 'prod_price': prod_prices_ls, 'prod_rating': prod_num_ratings_ls,
                 'prod_sales': prod_num_sales_ls,  'prod_details': prod_details_ls, 'main_cat': cat_name})

                     
            

            theme_df = pd.concat([theme_df, temp_df], axis = 0)       

    os.chdir('/home/valesco/Datasets/business_data/theme_forest_scrapes/')

    now = datetime.now().strftime('%Y-%d-%m')
    file_name = 'theme_forest_scrape_' + cat_name + '_' + now + '.csv'

    theme_df.to_csv(file_name, index = False, sep = '|')
    
browser.close()

  0%|          | 0/60 [00:00<?, ?it/s]

Starting iteration 1 for category site-templates now!


 28%|██▊       | 17/60 [00:59<02:19,  3.24s/it]

In [ ]:
category_url = 'https://themeforest.net/category'
ua = UserAgent()

headers = {
    'User-Agent': ua.random,
    'Content-Type': 'text/html'}

response_cat = requests.get(category_url, headers = headers, timeout = 1)

cookies = response_cat.cookies
soup = bs(response_cat.content, 'lxml')

theme_df

In [478]:
## Create Main DF

theme_df = pd.DataFrame(columns = ['datetime', 'prod_name', 'sort_type', 'prod_rank', 'prod_url', 'prod_author', 
                                  'prod_cat', 'prod_price', 'prod_rating', 'prod_sales', 'prod_details', 
                                  'main_cat'])
    
## Retrieve categories from base url

first_categories = soup.find_all(attrs = {'id': 'first'})
links = first_categories[0].find_all('a')


## Append links to list
category_url_ls = []
category_name_ls = []
for link in links:
    if link.get('href').split('/')[-2] == 'category':
        category_url_ls.append(link.get('href'))
        category_name_ls.append(link.get('href').replace('/category/',''))
        
category_name_ls

['site-templates',
 'wordpress',
 'cms-themes',
 'ecommerce',
 'blogging',
 'marketing',
 'forums',
 'psd-templates',
 'muse-templates',
 'sketch-templates',
 'typeengine-themes',
 'static-site-generators',
 'courses']

In [481]:
#temp_df = pd.DataFrame({'prod_name': prod_names_ls, 'sort_type': 'sales', 'prod_rank': prod_ranks_ls, 
#                       'prod_url': prod_urls_ls, 'prod_author': prod_authors_ls, 'prod_cat': prod_category_ls,
#                       'prod_price': prod_prices_ls, 'prod_rating': prod_num_ratings_ls,
#                       'prod_sales': prod_num_sales_ls,  'prod_details': prod_details_ls})

os.chdir('/home/valesco/Datasets/business_data/theme_forest_scrapes/')

now = datetime.now().strftime('%Y-%d-%m %H:%M')

with open('theme_forest_scrape_' + now + '.json', 'w') as outfile:
    json.dump(master_ls, outfile)

In [57]:
theme_df = pd.DataFrame(columns = ['datetime', 'prod_name', 'sort_type', 'prod_rank', 'prod_url', 'prod_author', 
                                  'prod_cat', 'prod_price', 'prod_rating', 'prod_sales', 'prod_details', 
                                  'main_cat'])

pd.concat([theme_df, temp_df], axis = 0)

,datetime,main_cat,prod_author,prod_cat,prod_details,prod_name,prod_price,prod_rank,prod_rating,prod_sales,prod_url,sort_type
0,2017-04-25 18:48:27.687620,courses,tutsplus,"[Courses, Code]","[Courses /, Closed Captions: No, Resolution: 1...",What's New in Rails 5?,5,121,,2,https://themeforest.net/item/whats-new-in-rail...,trending
1,2017-04-25 18:48:27.687620,courses,tutsplus,"[Courses, Code]","[Courses /, Closed Captions: No, Resolution: 1...",Getting Started With Django,15,122,,7,https://themeforest.net/item/getting-started-w...,trending
2,2017-04-25 18:48:27.687620,courses,tutsplus,"[Courses, Code]","[Courses /, Closed Captions: No, Resolution: 1...",Connect the Web With WebSockets,5,123,,7,https://themeforest.net/item/connect-the-web-w...,trending
3,2017-04-25 18:48:27.687620,courses,tutsplus,"[Courses, Web Design]","[Courses /, Web Design, Closed Captions: No,...",Workshop Your Way Through the Web Design Process,5,124,,18,https://themeforest.net/item/workshop-your-way...,trending
4,2017-04-25 18:48:27.687620,courses,tutsplus,"[Courses, Code]","[Courses /, Closed Captions: No, Resolution: 1...",Building a CMS With ASP.NET MVC5,15,125,,6,https://themeforest.net/item/building-a-cms-wi...,trending
5,2017-04-25 18:48:27.687620,courses,tutsplus,"[Courses, Web Design]","[Courses /, Web Design, Closed Captions: No,...",Building Static Websites With Jekyll,15,126,,5,https://themeforest.net/item/building-static-w...,trending
6,2017-04-25 18:48:27.687620,courses,tutsplus,"[Courses, Code]","[Courses /, Closed Captions: No, Resolution: 1...",Improving C# With Version 6,5,127,,,https://themeforest.net/item/improving-c-with-...,trending
7,2017-04-25 18:48:27.687620,courses,tutsplus,"[Courses, Code]","[Courses /, Closed Captions: No, Resolution: 1...",Deploy Your Rails Application Into Heroku,5,128,,,https://themeforest.net/item/deploy-your-rails...,trending
8,2017-04-25 18:48:27.687620,courses,tutsplus,"[Courses, Code]","[Courses /, Closed Captions: No, Resolution: 1...",Get Started with ASP.NET MVC 6,15,129,,5,https://themeforest.net/item/get-started-with-...,trending


In [344]:
prod_details = cat_soup.find_all(attrs = {'class': 'product-list__column-category'})

## retrieve product details
prod_details_ls = []
for prod_detail in prod_details:
    for br, br_tag in enumerate(prod_detail.find('p').getText().split('\n')):
        if len(br_tag) > 30:
            print(br_tag)
            prod_details_ls.append(br_tag.replace('    ', ''))
            
len(prod_details_ls)

0

In [257]:
temp = cat_soup.find('div', attrs = {'class': 'content-l -size-scale-tablet content-right'})
data_view = temp.find_all('div', attrs = {'data-view': 'productList'})
data_view2 = data_view[0].find_all(attrs = {'data-view': 'bookmarkStatesLoader'})
data_view3 = data_view2[0].find_all(attrs = {'class': 'js-google-analytics__list-event-container'})
container = cat_soup.find_all(attrs = {'class': 'product-list__heading'})

In [54]:
theme_df =

,datetime,main_cat,prod_author,prod_cat,prod_details,prod_name,prod_price,prod_rank,prod_rating,prod_sales,prod_url,sort_type
0,2017-04-25 18:48:27.687620,courses,tutsplus,"[Courses, Code]","[Courses /, Closed Captions: No, Resolution: 1...",What's New in Rails 5?,5,121,,2,https://themeforest.net/item/whats-new-in-rail...,trending
1,2017-04-25 18:48:27.687620,courses,tutsplus,"[Courses, Code]","[Courses /, Closed Captions: No, Resolution: 1...",Getting Started With Django,15,122,,7,https://themeforest.net/item/getting-started-w...,trending
2,2017-04-25 18:48:27.687620,courses,tutsplus,"[Courses, Code]","[Courses /, Closed Captions: No, Resolution: 1...",Connect the Web With WebSockets,5,123,,7,https://themeforest.net/item/connect-the-web-w...,trending
3,2017-04-25 18:48:27.687620,courses,tutsplus,"[Courses, Web Design]","[Courses /, Web Design, Closed Captions: No,...",Workshop Your Way Through the Web Design Process,5,124,,18,https://themeforest.net/item/workshop-your-way...,trending
4,2017-04-25 18:48:27.687620,courses,tutsplus,"[Courses, Code]","[Courses /, Closed Captions: No, Resolution: 1...",Building a CMS With ASP.NET MVC5,15,125,,6,https://themeforest.net/item/building-a-cms-wi...,trending
5,2017-04-25 18:48:27.687620,courses,tutsplus,"[Courses, Web Design]","[Courses /, Web Design, Closed Captions: No,...",Building Static Websites With Jekyll,15,126,,5,https://themeforest.net/item/building-static-w...,trending
6,2017-04-25 18:48:27.687620,courses,tutsplus,"[Courses, Code]","[Courses /, Closed Captions: No, Resolution: 1...",Improving C# With Version 6,5,127,,,https://themeforest.net/item/improving-c-with-...,trending
7,2017-04-25 18:48:27.687620,courses,tutsplus,"[Courses, Code]","[Courses /, Closed Captions: No, Resolution: 1...",Deploy Your Rails Application Into Heroku,5,128,,,https://themeforest.net/item/deploy-your-rails...,trending
8,2017-04-25 18:48:27.687620,courses,tutsplus,"[Courses, Code]","[Courses /, Closed Captions: No, Resolution: 1...",Get Started with ASP.NET MVC 6,15,129,,5,https://themeforest.net/item/get-started-with-...,trending


In [52]:
temp_dict = temp_df.T.to_dict()
master_ls = []
master_ls.append(temp_dict)

pd.DataFrame.from_records(master_ls)


,0,1,2,3,4,5,6,7,8
0,"{'prod_author': 'tutsplus', 'prod_price': '5',...","{'prod_author': 'tutsplus', 'prod_price': '15'...","{'prod_author': 'tutsplus', 'prod_price': '5',...","{'prod_author': 'tutsplus', 'prod_price': '5',...","{'prod_author': 'tutsplus', 'prod_price': '15'...","{'prod_author': 'tutsplus', 'prod_price': '15'...","{'prod_author': 'tutsplus', 'prod_price': '5',...","{'prod_author': 'tutsplus', 'prod_price': '5',...","{'prod_author': 'tutsplus', 'prod_price': '15'..."


In [455]:
os.chdir('/home/valesco/Datasets/business_data/theme_forest_scrapes/')
now = datetime.now().strftime('%Y-%d-%m %H:%M')

theme_df.to_csv('theme_forest_scrape_' + now + '.csv', index = False, sep = '|')

In [456]:
category_name_ls[8]

'muse-templates'

In [62]:
cat_page_url

'https://themeforest.net/category/site-templates?page=1&referrer=search&sort=price-asc&utf8=✓&view=list'